## Multilayer Perceptron

### Feed forward e backpropagation

Esta tarefa visa implementar uma rede neural totalmente conectada (multilayer perceptron) para uma tarefa simples (predição de saída de portas lógicas).

A rede neural deve implementar a função $X_1 \text{ AND } (X_2 \text{ OR } X_3)$

Teste você mesmo no WolframAlpha: http://www.wolframalpha.com/input/?i=(b+OR+c)+AND+a

In [1]:
import numpy as np
import random
import pandas as pd
import copy
from sklearn.metrics import accuracy_score
from IPython.display import display

X = np.array([
    [0, 0, 0],
    [0, 0, 1],
    [0, 1, 0],
    [0, 1, 1],
    [1, 0, 0],
    [1, 0, 1],
    [1, 1, 0],
    [1, 1, 1]
])

Y = np.array([
    [0],
    [0],
    [0],
    [0],
    [0],
    [1],
    [1],
    [1]
])

weights = [
    np.array([
        [0.64, 0.22], 
        [0.15, 0.71],
        [0.27, 0.16],
        [0.13, 0.56]
    ]),
    np.array([
        [0.65],
        [0.19],
        [0.25]
    ])
]

lr = 1.
layer_configuration = [2]
n_instances, n_attributes = X.shape
n_classes = len(np.unique(Y))
n_epochs = 10000

## Tarefa 1: Implemente a função sigmoide.

In [2]:
def sigmoid(Z):
    """
    Função sigmoide para uma ativação linear Z.

    :param Z: Ativação linear.
    :return: sigmoide da ativação linear.
    """
    value = None
    ### BEGIN SOLUTION
    value = 1./(1. + (np.e ** -Z))
    ### END SOLUTION
    return value

In [3]:
"""Testes para a Tarefa 1"""

from nose.tools import *

a = np.linspace(start=0, stop=1, num=10, dtype=np.float32)
correct = np.array([0.5, 0.53, 0.56, 0.58, 0.61, 0.64, 0.66, 0.69, 0.71, 0.73], dtype=np.float32)
res = sigmoid(a)

ok_(np.all(abs(correct - res) < 0.1))

## Tarefa 2: Implemente a derivada da função sigmoide.

In [4]:
def d_sigmoid(L):
    """
    Derivada da função sigmoide.

    :param L: Valor de ativação da função sigmoide.
    :return: A derivada da função sigmoide.
    """
    value = None
    ### BEGIN SOLUTION
    value = (1. - L) * L
    ### END SOLUTION
    return value


In [5]:
"""Testes para a Tarefa 2"""

from nose.tools import *

a = np.array([0.5, 0.53, 0.56, 0.58, 0.61, 0.64, 0.66, 0.69, 0.71, 0.73], dtype=np.float32)
correct = np.array([0.25, 0.25, 0.25, 0.24, 0.24, 0.23, 0.22, 0.21, 0.21, 0.20], dtype=np.float32)
res = d_sigmoid(a)

ok_(np.all(abs(correct - res) < 0.1))

## Tarefa 3: Dado um conjunto de pesos, inicie a estrutura de uma rede neural totalmente conectada.

In [6]:
class MLP_trial(object):
    def __init__(self, layer_configuration, n_attributes, n_classes, lr=1., weights=None):
        """
        Constrói uma rede neural Multilayer Perceptron (MLP) com os parâmetros especificados.

        :param layer_configuration: Uma lista onde cada entrada é uma camada escondida,
            e cada número o número de neurônios naquela camada escondida. Não é necessário passar as dimensões
            das camadas de entrada e saída; elas devem ser computadas automaticamente.
        :type layer_configuration: list
        :param n_attributes: número de atributos (i.e. colunas) do dataset
        :type n_attributes: int
        :param n_classes: número de classes (i.e. rótulos) do dataset
        :type n_classes: int
        :param lr: (opcional) taxa de aprendizado. Valor padrão é 1 (não ameniza o erro dos neurônios)
        :type lr: float
        :param weights: (opcional): caso você deseje iniciar uma rede neural com pesos pré-configurados,
            passe a lista de pesos para este parâmetro. Do contrário, os pesos precisam ser iniciados 
            randomicamente entre [0.1, 1)
        :type weights: list
        """

        # camada de entrada deve ter dimensões N x (M + 1), onde:
        # N é o número de instâncias do dataset
        # M é o número de atributos do dataset
        # +1 é adicionado a M para acomodar o bias

        self.layer_configuration = layer_configuration
        self.n_attributes = n_attributes
        self.n_classes = n_classes
        self.lr = float(lr)
        self.n_layers = len(self.layer_configuration) + 1  # +1 para fechar a camada de saída

        # constrói as camadas
        if weights is not None:
            self.weights = copy.deepcopy(weights)  # copia os pesos
        else:
            self.weights = []

            ### BEGIN SOLUTION
            for j in range(self.n_layers):
                if j == 0:  # camada de entrada é um caso especial
                    dims = (n_attributes + 1, self.layer_configuration[j])
                elif j == (self.n_layers - 1):  # camada de saída é um caso especial
                    dims = (self.layer_configuration[j - 1] + 1, self.n_classes if self.n_classes > 2 else 1)
                else:  # camada escondida
                    dims = (self.layer_configuration[j - 1] + 1, self.layer_configuration[j])

                exec('self.weights += [np.random.random(%s) + 0.1]' % str(dims))
            ### END SOLUTION

In [7]:
"""Testes para a Tarefa 3"""

from nose.tools import *

# Teste 1: Checa se os pesos foram atribuídos corretamente

clf1 = MLP_trial(
    layer_configuration=layer_configuration, 
    n_attributes=n_attributes, 
    n_classes=n_classes, 
    lr=lr, 
    weights=weights 
)

# checa se o modelo tem o mesmo número 
# de camadas que os pesos passados
ok_(len(clf1.weights) == len(weights))

for i in range(len(clf1.weights)):  # itera sobre as camadas
    ok_(np.all(abs(clf1.weights[i]) - abs(weights[i]) < 0.1))  # checa se os pesos da camada são iguais

# Teste 2: Checa se a classe é capaz de instanciar pesos randômicos corretamente

clf2 = MLP_trial(
    layer_configuration=layer_configuration, 
    n_attributes=n_attributes, 
    n_classes=n_classes, 
    lr=lr
)

# checa se o modelo tem o número correto de camadas
ok_(len(clf2.weights) == len(weights))

for i in range(len(clf2.weights)):  # itera sobre as camadas
    ok_(np.all(clf2.weights[i].shape == weights[i].shape))  # checa se as dimensões dos pesos nesta camada estão corretas

## Tarefa 4: Faça o feedforward da rede neural.

In [8]:
def predict(self, X):
    """
    Realize predições para um conjunto de instâncias X.

    :param X: Instâncias para realizar predição.
    :return: Predições (arredondadas) para as instâncias X.
    """
    predictions = np.empty(len(X), dtype=np.float32)

    # os pesos da rede estão armazenados no atributo self.weights
    # print(self.weights)
    
    ### BEGIN SOLUTION
    if X.ndim == 1:
        X = X.reshape(X.shape[0], 1)

    n_pred = X.shape[0]

    L = X
    for W in self.weights:
        # adiciona o bias
        L = np.hstack((
            np.ones((n_pred, 1)), L
        ))

        L = self.sigmoid(L.dot(W))

    if self.n_classes == 2:
        predictions = np.round(L)
    else:
        predictions = np.argmax(L, axis=0).astype(np.float32)
    ### END SOLUTION
    return predictions

In [9]:
"""Testes para a Tarefa 4"""

from nose.tools import *
import types

clf = MLP_trial(
    layer_configuration=layer_configuration, 
    n_attributes=n_attributes, 
    n_classes=n_classes, 
    lr=lr, 
    weights=weights 
)

# injeta a função sigmoide APENAS nesta instância
clf.sigmoid = sigmoid
# injeta a função predict APENAS nesta instância
clf.predict = types.MethodType(predict, clf)

predictions = clf.predict(X)
ok_(np.all(abs(predictions) - 1) < 0.01)

## Tarefa 5: Calcule a função de custo entropia cruzada binária.

In [10]:
def cost_function(H, Y):
    """
    Implementa a função de custo para a ativação sigmoide.

    :param H: Array com predições.
    :param Y: Array com valores reais das classes.
    :return: O custo da predição atual.
    """
    n_instances = H.shape[0]
    
    cost = np.float32(1.)
    
    ### BEGIN SOLUTION
    _Y = Y.ravel()
    _H = H.ravel()
    
    soma = (_Y * np.log2(_H)) + (1 - _Y) * np.log2(1 - _H)

    cost = - (1. / n_instances) * soma.sum()
    ### END SOLUTION
    return cost

In [11]:
"""Testes para a Tarefa 5"""

from nose.tools import *

# cria predições randômicas
y_pred = np.array([0.41, 0.47, 0.27, 0.29, 0.46, 0.86, 0.59, 0.28], dtype=np.float32)
correct_cost = 0.79  # custo de predição para as instâncias y_pred

ok_(abs(cost_function(y_pred, Y) - correct_cost) < 0.01)

## Tarefa 6: Com base em todas as informações anteriores, implemente o treinamento de uma rede neural totalmente conectada com backpropagation.

In [12]:
class MLP(MLP_trial):
    def __init__(self, *args, **kwargs):
        super(MLP, self).__init__(*args, **kwargs)

    def fit(self, X, Y, n_epochs):
        """
        Realiza um treinamento sobre os dados de treino X.

        :param X: Atributos preditivos para treino.
        :param Y: Atributo classe para o treino.
        :param n_epochs: Número de épocas para realizar o treinamento.
        """
        N = X.shape[0]  # número de instâncias de treino
        X = X.astype(np.float32)

        for epoch in range(n_epochs):
            ### BEGIN SOLUTION
            Ls = [X]  # ativações
            L = Ls[0]
            for W in self.weights:
                L = np.hstack((np.ones((len(L), 1), dtype=np.float32), L))  # adiciona bias, que é sempre 1

                L = self.sigmoid(L.dot(W))
                Ls += [L]

            n_layers = len(self.weights)  # número de camadas

            # calcula os gradientes
            Gs = [np.array([[1.]])]  # o último gradiente é sempre 1, porque df/df = 1
            Gs += [(Ls[-1] - Y) * self.d_sigmoid(Ls[-1]) * self.lr]
            # n_layers - 1 porque o primeiro parâmetro é inclusivo, e o último exclusivo
            for desc in range(n_layers - 1, 0, -1):
                G = Gs[-1]
                L = Ls[desc]
                W = self.weights[desc]

                L = np.hstack((
                    np.ones((len(L), 1), dtype=np.float32), L
                ))

                _G = G.dot(W.T) * self.d_sigmoid(L)
                _G = _G[:, 1:]

                Gs += [_G]

            # inverte a lista de gradientes, para que fiquem na ordem correta
            # (posição 0 tem o gradiente da primeira camada, e assim por diante)
            Gs = Gs[::-1]

            for asc in range(n_layers):
                L = Ls[asc]
                G = Gs[asc]

                L = np.hstack((
                    np.ones((len(L), 1), dtype=np.float32), L
                ))

                self.weights[asc] -= L.T.dot(G)
            ### END SOLUTION
            print('Erro para a época %4.d: %.6f' % (epoch, self.cost_function(Ls[-1], Y)))

In [13]:
"""Testes para a Tarefa 6"""

from nose.tools import *
import types

clf = MLP(
    layer_configuration=layer_configuration, 
    n_attributes=n_attributes, 
    n_classes=n_classes, 
    lr=lr, 
    weights=weights 
)

# injeta funções nesta classe
clf.sigmoid = sigmoid
clf.predict = types.MethodType(predict, clf)
clf.d_sigmoid = d_sigmoid
clf.cost_function = cost_function

clf.fit(X, Y, n_epochs=n_epochs)

Erro para a época    0: 1.319421
Erro para a época    1: 0.988732
Erro para a época    2: 0.968035
Erro para a época    3: 0.965324
Erro para a época    4: 0.962686
Erro para a época    5: 0.960082
Erro para a época    6: 0.957483
Erro para a época    7: 0.954858
Erro para a época    8: 0.952172
Erro para a época    9: 0.949392
Erro para a época   10: 0.946479
Erro para a época   11: 0.943394
Erro para a época   12: 0.940091
Erro para a época   13: 0.936524
Erro para a época   14: 0.932638
Erro para a época   15: 0.928373
Erro para a época   16: 0.923663
Erro para a época   17: 0.918431
Erro para a época   18: 0.912595
Erro para a época   19: 0.906063
Erro para a época   20: 0.898734
Erro para a época   21: 0.890501
Erro para a época   22: 0.881259
Erro para a época   23: 0.870903
Erro para a época   24: 0.859346
Erro para a época   25: 0.846527
Erro para a época   26: 0.832426
Erro para a época   27: 0.817073
Erro para a época   28: 0.800557
Erro para a época   29: 0.783024
Erro para 

Erro para a época  867: 0.025106
Erro para a época  868: 0.025086
Erro para a época  869: 0.025066
Erro para a época  870: 0.025045
Erro para a época  871: 0.025025
Erro para a época  872: 0.025005
Erro para a época  873: 0.024985
Erro para a época  874: 0.024964
Erro para a época  875: 0.024944
Erro para a época  876: 0.024924
Erro para a época  877: 0.024904
Erro para a época  878: 0.024884
Erro para a época  879: 0.024864
Erro para a época  880: 0.024844
Erro para a época  881: 0.024825
Erro para a época  882: 0.024805
Erro para a época  883: 0.024785
Erro para a época  884: 0.024765
Erro para a época  885: 0.024746
Erro para a época  886: 0.024726
Erro para a época  887: 0.024706
Erro para a época  888: 0.024687
Erro para a época  889: 0.024667
Erro para a época  890: 0.024648
Erro para a época  891: 0.024628
Erro para a época  892: 0.024609
Erro para a época  893: 0.024590
Erro para a época  894: 0.024570
Erro para a época  895: 0.024551
Erro para a época  896: 0.024532
Erro para 

Erro para a época 1777: 0.015538
Erro para a época 1778: 0.015533
Erro para a época 1779: 0.015527
Erro para a época 1780: 0.015522
Erro para a época 1781: 0.015516
Erro para a época 1782: 0.015511
Erro para a época 1783: 0.015505
Erro para a época 1784: 0.015499
Erro para a época 1785: 0.015494
Erro para a época 1786: 0.015488
Erro para a época 1787: 0.015483
Erro para a época 1788: 0.015477
Erro para a época 1789: 0.015472
Erro para a época 1790: 0.015466
Erro para a época 1791: 0.015461
Erro para a época 1792: 0.015455
Erro para a época 1793: 0.015450
Erro para a época 1794: 0.015444
Erro para a época 1795: 0.015439
Erro para a época 1796: 0.015433
Erro para a época 1797: 0.015428
Erro para a época 1798: 0.015422
Erro para a época 1799: 0.015417
Erro para a época 1800: 0.015412
Erro para a época 1801: 0.015406
Erro para a época 1802: 0.015401
Erro para a época 1803: 0.015395
Erro para a época 1804: 0.015390
Erro para a época 1805: 0.015384
Erro para a época 1806: 0.015379
Erro para 

Erro para a época 2639: 0.012135
Erro para a época 2640: 0.012132
Erro para a época 2641: 0.012129
Erro para a época 2642: 0.012126
Erro para a época 2643: 0.012124
Erro para a época 2644: 0.012121
Erro para a época 2645: 0.012118
Erro para a época 2646: 0.012115
Erro para a época 2647: 0.012112
Erro para a época 2648: 0.012110
Erro para a época 2649: 0.012107
Erro para a época 2650: 0.012104
Erro para a época 2651: 0.012101
Erro para a época 2652: 0.012098
Erro para a época 2653: 0.012096
Erro para a época 2654: 0.012093
Erro para a época 2655: 0.012090
Erro para a época 2656: 0.012087
Erro para a época 2657: 0.012084
Erro para a época 2658: 0.012082
Erro para a época 2659: 0.012079
Erro para a época 2660: 0.012076
Erro para a época 2661: 0.012073
Erro para a época 2662: 0.012070
Erro para a época 2663: 0.012068
Erro para a época 2664: 0.012065
Erro para a época 2665: 0.012062
Erro para a época 2666: 0.012059
Erro para a época 2667: 0.012057
Erro para a época 2668: 0.012054
Erro para 

Erro para a época 3547: 0.010143
Erro para a época 3548: 0.010141
Erro para a época 3549: 0.010139
Erro para a época 3550: 0.010138
Erro para a época 3551: 0.010136
Erro para a época 3552: 0.010134
Erro para a época 3553: 0.010133
Erro para a época 3554: 0.010131
Erro para a época 3555: 0.010129
Erro para a época 3556: 0.010128
Erro para a época 3557: 0.010126
Erro para a época 3558: 0.010124
Erro para a época 3559: 0.010122
Erro para a época 3560: 0.010121
Erro para a época 3561: 0.010119
Erro para a época 3562: 0.010117
Erro para a época 3563: 0.010116
Erro para a época 3564: 0.010114
Erro para a época 3565: 0.010112
Erro para a época 3566: 0.010110
Erro para a época 3567: 0.010109
Erro para a época 3568: 0.010107
Erro para a época 3569: 0.010105
Erro para a época 3570: 0.010104
Erro para a época 3571: 0.010102
Erro para a época 3572: 0.010100
Erro para a época 3573: 0.010099
Erro para a época 3574: 0.010097
Erro para a época 3575: 0.010095
Erro para a época 3576: 0.010094
Erro para 

...

Erro para a época 6119: 0.007356
Erro para a época 6120: 0.007355
Erro para a época 6121: 0.007355
Erro para a época 6122: 0.007354
Erro para a época 6123: 0.007353
Erro para a época 6124: 0.007353
Erro para a época 6125: 0.007352
Erro para a época 6126: 0.007351
Erro para a época 6127: 0.007350
Erro para a época 6128: 0.007350
Erro para a época 6129: 0.007349
Erro para a época 6130: 0.007348
Erro para a época 6131: 0.007348
Erro para a época 6132: 0.007347
Erro para a época 6133: 0.007346
Erro para a época 6134: 0.007346
Erro para a época 6135: 0.007345
Erro para a época 6136: 0.007344
Erro para a época 6137: 0.007344
Erro para a época 6138: 0.007343
Erro para a época 6139: 0.007342
Erro para a época 6140: 0.007341
Erro para a época 6141: 0.007341
Erro para a época 6142: 0.007340
Erro para a época 6143: 0.007339
Erro para a época 6144: 0.007339
Erro para a época 6145: 0.007338
Erro para a época 6146: 0.007337
Erro para a época 6147: 0.007337
Erro para a época 6148: 0.007336
Erro para 

Erro para a época 7013: 0.006799
Erro para a época 7014: 0.006798
Erro para a época 7015: 0.006798
Erro para a época 7016: 0.006797
Erro para a época 7017: 0.006796
Erro para a época 7018: 0.006796
Erro para a época 7019: 0.006795
Erro para a época 7020: 0.006795
Erro para a época 7021: 0.006794
Erro para a época 7022: 0.006794
Erro para a época 7023: 0.006793
Erro para a época 7024: 0.006792
Erro para a época 7025: 0.006792
Erro para a época 7026: 0.006791
Erro para a época 7027: 0.006791
Erro para a época 7028: 0.006790
Erro para a época 7029: 0.006790
Erro para a época 7030: 0.006789
Erro para a época 7031: 0.006789
Erro para a época 7032: 0.006788
Erro para a época 7033: 0.006787
Erro para a época 7034: 0.006787
Erro para a época 7035: 0.006786
Erro para a época 7036: 0.006786
Erro para a época 7037: 0.006785
Erro para a época 7038: 0.006785
Erro para a época 7039: 0.006784
Erro para a época 7040: 0.006784
Erro para a época 7041: 0.006783
Erro para a época 7042: 0.006782
Erro para 

Erro para a época 7840: 0.006377
Erro para a época 7841: 0.006376
Erro para a época 7842: 0.006376
Erro para a época 7843: 0.006376
Erro para a época 7844: 0.006375
Erro para a época 7845: 0.006375
Erro para a época 7846: 0.006374
Erro para a época 7847: 0.006374
Erro para a época 7848: 0.006373
Erro para a época 7849: 0.006373
Erro para a época 7850: 0.006372
Erro para a época 7851: 0.006372
Erro para a época 7852: 0.006371
Erro para a época 7853: 0.006371
Erro para a época 7854: 0.006370
Erro para a época 7855: 0.006370
Erro para a época 7856: 0.006370
Erro para a época 7857: 0.006369
Erro para a época 7858: 0.006369
Erro para a época 7859: 0.006368
Erro para a época 7860: 0.006368
Erro para a época 7861: 0.006367
Erro para a época 7862: 0.006367
Erro para a época 7863: 0.006366
Erro para a época 7864: 0.006366
Erro para a época 7865: 0.006365
Erro para a época 7866: 0.006365
Erro para a época 7867: 0.006364
Erro para a época 7868: 0.006364
Erro para a época 7869: 0.006363
Erro para 

Erro para a época 8729: 0.005997
Erro para a época 8730: 0.005997
Erro para a época 8731: 0.005996
Erro para a época 8732: 0.005996
Erro para a época 8733: 0.005996
Erro para a época 8734: 0.005995
Erro para a época 8735: 0.005995
Erro para a época 8736: 0.005995
Erro para a época 8737: 0.005994
Erro para a época 8738: 0.005994
Erro para a época 8739: 0.005993
Erro para a época 8740: 0.005993
Erro para a época 8741: 0.005993
Erro para a época 8742: 0.005992
Erro para a época 8743: 0.005992
Erro para a época 8744: 0.005991
Erro para a época 8745: 0.005991
Erro para a época 8746: 0.005991
Erro para a época 8747: 0.005990
Erro para a época 8748: 0.005990
Erro para a época 8749: 0.005989
Erro para a época 8750: 0.005989
Erro para a época 8751: 0.005989
Erro para a época 8752: 0.005988
Erro para a época 8753: 0.005988
Erro para a época 8754: 0.005987
Erro para a época 8755: 0.005987
Erro para a época 8756: 0.005987
Erro para a época 8757: 0.005986
Erro para a época 8758: 0.005986
Erro para 

Erro para a época 9644: 0.005667
Erro para a época 9645: 0.005666
Erro para a época 9646: 0.005666
Erro para a época 9647: 0.005666
Erro para a época 9648: 0.005665
Erro para a época 9649: 0.005665
Erro para a época 9650: 0.005665
Erro para a época 9651: 0.005664
Erro para a época 9652: 0.005664
Erro para a época 9653: 0.005664
Erro para a época 9654: 0.005663
Erro para a época 9655: 0.005663
Erro para a época 9656: 0.005663
Erro para a época 9657: 0.005662
Erro para a época 9658: 0.005662
Erro para a época 9659: 0.005662
Erro para a época 9660: 0.005661
Erro para a época 9661: 0.005661
Erro para a época 9662: 0.005661
Erro para a época 9663: 0.005660
Erro para a época 9664: 0.005660
Erro para a época 9665: 0.005660
Erro para a época 9666: 0.005659
Erro para a época 9667: 0.005659
Erro para a época 9668: 0.005659
Erro para a época 9669: 0.005658
Erro para a época 9670: 0.005658
Erro para a época 9671: 0.005658
Erro para a época 9672: 0.005657
Erro para a época 9673: 0.005657
Erro para 

## Tarefa 7: Obtenha 100% de acurácia no conjunto de treino.

* Neste exercício não existe conjunto de teste; todos os dados possíveis estão no conjunto de treino.
* O objetivo é aproximar corretamente a função $X_1 \text{ AND } (X_2 \text{ OR } X_3)$


In [14]:
"""Testes para a Tarefa 7"""

from nose.tools import *

pred = clf.predict(X)  # prediz o próprio conjunto de treino; apenas para verificar se está aprendendo

acc = accuracy_score(Y, pred)
print('Acurácia no conjunto de treino:', acc)

ok_(abs(1. - acc) < 0.01)

Acurácia no conjunto de treino: 1.0
